In [ ]:
%pip install sunpy astropy sherpa matplotlib -q

from pathlib import Path
import zipfile, gzip
from io import BytesIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import Markdown

from astropy.io import fits
from astropy.table import Table
from astropy.time import Time

# import sherpa

# import sunpy

In [ ]:
# Find the latest Aditya-L1 zip
data_dir     = Path('data')
source_files = sorted(data_dir.glob('AL1_SLX_L1_*_v1.0.zip'))
if not source_files:
    raise FileNotFoundError("No ZIP files found in data/")
print("Most recent data file:", source_files[-1].name, end='\n\n')

# Locate the SDD2 light-curve gz inside that zip
with zipfile.ZipFile(source_files[-1], 'r') as z:
    members = [m for m in z.namelist() if m.endswith('.lc.gz') and '/SDD2/' in m]
    if not members:
        raise FileNotFoundError(f"No SDD2 .lc.gz found in {source_files[-1].name}")

    # Decompress & open with FITS
    raw = z.read(members[0])
    with gzip.GzipFile(fileobj=BytesIO(raw)) as dec:
        hdul = fits.open(dec)
        hdul.info()

        # Render HDU0 header
        hdr0 = hdul[0].header
        df0 = pd.DataFrame({
            "Keyword": list(hdr0.keys()),
            "Value":   [hdr0[k] for k in hdr0.keys()],
            "Comment": [hdr0.comments[k] for k in hdr0.keys()]
        })
        display(Markdown("### Primary HDU Header\n" + df0.to_markdown(index=False)))

        # Render HDU1 (table) header
        hdr1 = hdul[1].header
        df1 = pd.DataFrame({
            "Keyword": list(hdr1.keys()),
            "Value":   [hdr1[k] for k in hdr1.keys()],
            "Comment": [hdr1.comments[k] for k in hdr1.keys()]
        })
        display(Markdown("### Table HDU Header\n" + df1.to_markdown(index=False)))

In [ ]:
parquet_file = Path('SoLEXS_dataset.parquet')
if parquet_file.exists():

    SoLEXS_df       = pd.read_parquet(parquet_file)
    processed_dates = SoLEXS_df['DATE'].dt.strftime('%Y%m%d').unique()
    for day in source_files.copy():
        if day.stem.split('_')[3] in processed_dates:
            source_files.remove(Path(day))

else:
    
    SoLEXS_df       = pd.DataFrame()

for zip_path in source_files:
    prefix = f"{zip_path.stem}/SDD2/"
    with zipfile.ZipFile(zip_path) as zf:
        for member in zf.namelist():
            if member.startswith(prefix) and member.endswith('.lc.gz'):
                raw = zf.read(member)
                with gzip.GzipFile(fileobj=BytesIO(raw)) as dec:
                    tbl = Table.read(dec, format='fits')
                    df  = tbl.to_pandas()
                    df['DATE'] = pd.to_datetime(zip_path.stem.split('_')[3], format='%Y%m%d')
                    df['TIME'] = df['TIME'].astype(int)
                    SoLEXS_df = pd.concat([SoLEXS_df, df], ignore_index=True)

SoLEXS_df   = SoLEXS_df[['DATE','TIME','COUNTS']]
SoLEXS_df.to_parquet(parquet_file, index=False)
print(f"Appended {len(source_files)} new days; Total rows now {len(SoLEXS_df)}")

display(SoLEXS_df)

In [ ]:
ADITYA_EPOCH_UNIX = 1693612800  # 2023-09-02T00:00:00 UTC

display(SoLEXS_df.describe().loc[['min', 'max'], ['DATE', 'TIME']])
display(SoLEXS_df.describe().loc[:, ['COUNTS']].T.astype(int))

In [ ]:
start_date = '2025-03-20'
end_date   = '2025-03-27'

df = SoLEXS_df[(SoLEXS_df['DATE'] >= start_date) & (SoLEXS_df['DATE'] <= end_date)].copy()
display(df.describe().loc[['min', 'max']])

gap = 60    # Seconds of inactivity to split events
sigma = 5   # Threshold = median + sigma * std

times = SoLEXS_df['TIME']
counts = SoLEXS_df['COUNTS']

# Return sorted unique unix times where counts exceed median + sigma*std.
med, std = np.nanmedian(counts), np.nanstd(counts)
mask = counts > (med + sigma*std)
spikes = np.unique(times[mask])

In [ ]:
# Cluster unix timestamps into events: 
# gaps > `gap` seconds → new event.
# Returns list of (t_start, t_end) tuples.
if spikes.size == 0:
    events = []
else:
    groups = [[spikes[0]]]
    for t in spikes[1:]:
        if t - groups[-1][-1] <= gap:
            groups[-1].append(t)
        else:
            groups.append([t])
    events = [(g[0], g[-1]) for g in groups]

if not events:
    print("No flares detected.")
else:
    for i, ev in enumerate(events, 1):
        t0, t1 = Time(ev[0], format='unix'), Time(ev[1], format='unix')
        # midnight of that day:
        mid = Time(t0.iso[:10]+'T00:00:00', format='isot', scale='utc')
        info = {'start_iso': t0.iso,
                'end_iso':   t1.iso,
                'start_sod': (t0 - mid).sec,
                'end_sod':   (t1 - mid).sec}
        print(f"Flare {i}: {info['start_iso']} → {info['end_iso']}")

In [ ]:
start_time = Time(df['TIME'].iloc[0], format='unix').to_datetime().strftime('%H:%M:%S')
end_time = Time(df['TIME'].iloc[-1], format='unix').to_datetime().strftime('%H:%M:%S')

plt_times = Time(df['TIME'], format='unix').to_datetime()
plt.figure(figsize=(30, 15), dpi=300)
plt.plot(plt_times, df['COUNTS'])
plt.gcf().autofmt_xdate()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d %H:%M'))
plt.grid(True)
plt.xlabel("Time", fontsize=16)
plt.ylabel("X-Ray Count", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title(f"SoLEXS X-ray Lightcurve\n\n{start_date} {start_time} to {end_date} {end_time}", fontsize=16, fontweight='bold')
plt.show()

In [ ]:
baseline = np.nanmedian(SoLEXS_df['COUNTS'])
std_dev = np.nanstd(SoLEXS_df['COUNTS'])
threshold = baseline + 5*std_dev

times = df['TIME'].values
# flare_indices = np.where(df['COUNTS'] > threshold)[0]
flare_times = Time(np.unique(times), format='unix', scale='utc')
t_ref = Time(times[0], format='unix', scale='utc')
flare_offsets = (flare_times - t_ref).sec

In [ ]:
peak_idx = np.nanargmax(df['COUNTS'])
window = 2400  # seconds around peak
start = peak_idx - window
end = peak_idx + window

time_subset = Time(df['TIME'][start:end], format='unix').to_datetime()

plt.figure(figsize=(30, 15), dpi=300)
plt.plot(time_subset, df['COUNTS'][start:end])
plt.grid(True)
plt.xlabel("Time", fontsize=16)
plt.ylabel("X-Ray Count", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title(f"Zoomed-in Solar Flare\n\n{df['DATE'].iloc[-1].strftime('%Y-%m-%d')} {time_subset[0].strftime('%H:%M')} to {time_subset[-1].strftime('%H:%M')}", fontsize=20, fontweight='bold')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.show()